In [1]:
import torch
import os 
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer, BitsAndBytesConfig
from transformers import FalconModel, FalconConfig
from peft import PeftModel, PeftConfig

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../model')
config_path = os.path.join(default_path, '../../config')

In [3]:
access_tok = '-'

In [4]:
config = AutoConfig.from_pretrained('ybelkada/falcon-7b-sharded-bf16', token=access_tok)
peft_model_id = 'dev7halo/falcon-7b-sharded-bf16-KoAlpaca'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    config._name_or_path,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config._name_or_path)

model = model.to('cuda')
model.eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (rotary_emb): FalconRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4544, out_features=4672, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
         

In [45]:
user_input = '한국은행의 역할과 기능에 대해서 설명해보시요'

In [46]:
inputs = tokenizer(user_input, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=512,
                             temperature=0.8,
                             top_p=0.95,)
                             # do_sample=True, 
                             # early_stopping=True,
                             # no_repeat_ngram_size=3)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


한국은행의 역할과 기능에 대해서 설명해보시요. 역할은 상업적인 결정에서 사용자가 생산, 인수, 운수 등을 하는 형태로 하는 것을 의미합니다. 즉, 한국은행은 생산과 인수의 상업 기초를 마련하고, 이에 따라 국제조약을 통해 생산에 필요한 인수운수와 국내외 거래에 필요한 국제거래 등을 위하여 생산과 인수를 촉진시키는 역할을 수행합니다. 이를 위해 한국은행은 국제거래에 큰 영향을 끼쳐서 국내거래에 유리한 자유와 자산을 국내거래에서 넘어서는 영향을 끼치며, 이를
